Attribute Information:

The data set contains raw process sensor data (i.e. without feature extraction) which are structured as matrices (tab-delimited) with the rows representing the cycles and the columns the data points within a cycle. The sensors involved are:

Sensor		Physical quantity		Unit		Sampling rate
PS1		Pressure			bar		100 Hz
PS2		Pressure			bar		100 Hz
PS3		Pressure			bar		100 Hz
PS4		Pressure			bar		100 Hz
PS5		Pressure			bar		100 Hz
PS6		Pressure			bar		100 Hz
EPS1		Motor power			W		100 Hz
FS1		Volume flow			l/min		10 Hz
FS2		Volume flow			l/min		10 Hz
TS1		Temperature			Â°C		1 Hz
TS2		Temperature			Â°C		1 Hz
TS3		Temperature			Â°C		1 Hz
TS4		Temperature			Â°C		1 Hz
VS1		Vibration			mm/s		1 Hz
CE		Cooling efficiency (virtual)	%		1 Hz
CP		Cooling power (virtual)		kW		1 Hz
SE		Efficiency factor		%		1 Hz



Has Missing Values?

No

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

#PS: Pressure, bar, 100 Hz --> 100 Messungen pro Sekunde
#EPS: Motor power, W, 100 Hz 
#FS: Volume flow, l/min, 10 Hz --> 10 Messungen pro Sekunde
#TS: Temperature, Celsius, 1 Hz --> 1 Messung pro Sekunde
#VS: Vibration, mm/s, 1Hz
#CE: Cooling efficiency (virtual), %, 1 Hz
#CP: Cooling power (virtual), kW, 1 Hz
#SE: Efficency factor, %, 1 Hz


class DataProcessor:
    def __init__(self, input_path, file_names):
        self.input_path = input_path
        self.file_names = file_names
        
    def read_files(self):
        self.data = {}
        print("Reading files...")
        for file in self.file_names:
            with open(self.input_path + file + '.txt', 'r') as f:
                self.data[file] = pd.read_csv(f, header=None, sep='\t')
        return self.data
    
    def print_shape(self):
        print("Files read:")
        for file in self.data:
            print(f"{file}: {self.data[file].shape}")
            
    def create_target_df(self):
        target_columns = ['Cooler_Condition', 'Valve_Condition', 
                        'Internal_Pump_Leakage', 'Hydraulic_Accumulator', 
                        'Stable_Flag']
        self.df_target = pd.DataFrame(self.data['target'], columns=target_columns)
        self.valve_condition = self.df_target['Valve_Condition']
        del self.data['target']
        return self.valve_condition

def process_data():
    input_path = "input_data/"
    file_names = [
        "ce", "cp", "eps1", "se", "vs1", 
        "fs1", "fs2", 
        "ps1", "ps2", "ps3", "ps4", "ps5", "ps6",
        "ts1", "ts2", "ts3", "ts4", "target"
    ]
    
    processor = DataProcessor(input_path, file_names)
    data = processor.read_files()
    processor.print_shape()
    #processor.create_target_df()
    #df_target = processor.valve_condition
    return data, df_target

data, df_target = process_data()

<h1> 1. Data Exploration </h1>

<h2>CE sensor (1 Hz) </h2>

In [ ]:
fig, ax = plt.subplots(figsize=(5, 3), dpi=100)

for i in range(data['ce'].shape[0] - 1):
    ax.plot(data['ce'].iloc[i], color='blue', linewidth=0.5)

ax.plot(data['ce'].iloc[-1], color='red', linewidth=1, label='Last Series')

ax.set_title('Cooling Efficency Data', fontsize=10)
plt.xlabel("Time [s]")
plt.ylabel("Cooling Efficency [%]")

plt.tight_layout()
plt.show()

<h2>CP Sensor (1 Hz)</h2>

In [ ]:
fig, ax = plt.subplots(figsize=(10, 4), dpi=100)

for i in range(data['cp'].shape[0] - 1):
    ax.plot(data['cp'].iloc[i], color='blue', linewidth=0.5)

ax.plot(data['cp'].iloc[-1], color='red', linewidth=1, label='Last Series')

ax.set_title('Cooling power (virtual) Data', fontsize=10)
plt.xlabel("Time [s]")
plt.ylabel("Cooling Power [kW]")

plt.tight_layout()
plt.show()

<h2>VS Sensor (1 Hz)</h2>

In [ ]:
fig, ax = plt.subplots(figsize=(10, 4), dpi=100)

for i in range(data['vs1'].shape[0] - 1):
    ax.plot(data['vs1'].iloc[i], color='blue', linewidth=0.5)

ax.plot(data['vs1'].iloc[-1], color='red', linewidth=1, label='Last Series')

ax.set_title('Vibration Data', fontsize=10)
plt.xlabel("Time [s]")
plt.ylabel("Vibration [mm/s]")

plt.tight_layout()
plt.show()

<h2>SE Sensor (1 Hz)</h2>

In [ ]:
fig, ax = plt.subplots(figsize=(10, 4), dpi=100)

for i in range(data['se'].shape[0] - 1):
    ax.plot(data['se'].iloc[i], color='blue', linewidth=0.5)

ax.plot(data['se'].iloc[-1], color='red', linewidth=1, label='Last Series')

ax.set_title('Efficency Factor Data', fontsize=10)
plt.xlabel("Time [s]")
plt.ylabel("Efficency Factor [%]")

plt.tight_layout()
plt.show()

<h2>TS Sensors (1 Hz)</h2>

In [ ]:
temp_sensors = ['ts1', 'ts2', 'ts3', 'ts4']

fig, axes = plt.subplots(2, 2, figsize=(15, 8), dpi=100)
axes = axes.flatten()

for idx, s in enumerate(temp_sensors):
    ax = axes[idx]
    for i in range(data[s].shape[0] - 1):
        ax.plot(data[s].iloc[i], color='blue', linewidth=0.5)
    ax.plot(data[s].iloc[-1], color='red', linewidth=1)
    
    ax.set_title(f"Temperature Data: {s}")
    ax.set_xlabel("Time [s]")
    ax.set_ylabel("Temperature [°C]")

plt.tight_layout()
plt.show()

<h2>FS Sensors (10 Hz)</h2>

In [ ]:
vf_sensors = ['fs1', 'fs2']

fig, axes = plt.subplots(1, 2, figsize=(15, 4), dpi=100)
axes = axes.flatten()

for idx, s in enumerate(vf_sensors):
    ax = axes[idx]
    for i in range(data[s].shape[0] - 1):
        ax.plot(data[s].iloc[i], color='blue', linewidth=0.5)
    ax.plot(data[s].iloc[-1], color='red', linewidth=1)
    
    ax.set_title(f"Volume Flow Data: {s}")
    ax.set_xlabel("Time [x 100 ms]")
    ax.set_ylabel("Volume Flow [l/min]")

plt.tight_layout()
plt.show()

<h2>EPS Sensor (100 Hz)</h2>

In [ ]:
fig, ax = plt.subplots(figsize=(10, 4), dpi=100)

for i in range(data['eps1'].shape[0] - 1):
    ax.plot(data['eps1'].iloc[i], color='blue', linewidth=0.5)

ax.plot(data['eps1'].iloc[-1], color='red', linewidth=1, label='Last Series')

ax.set_title('Motor power Data', fontsize=10)
plt.xlabel("Time [x 10 ms]")
plt.ylabel("Motor power [W]")

plt.tight_layout()
plt.show()

<h2>PS Sensors (100 Hz)</h2>

In [ ]:
ps_sensors = ['ps1', 'ps2', 'ps3', 'ps4', 'ps5', 'ps6']

fig, axes = plt.subplots(3, 2, figsize=(15, 10), dpi=100)
axes = axes.flatten()

for idx, s in enumerate(ps_sensors):
    ax = axes[idx]
    for i in range(data[s].shape[0] - 1):
        ax.plot(data[s].iloc[i], color='blue', linewidth=0.5)
    ax.plot(data[s].iloc[-1], color='red', linewidth=1)
    
    ax.set_title(f"Pressure Data: {s}")
    ax.set_xlabel("Time [x 10 ms]")
    ax.set_ylabel("Pressure [Bar]")

plt.tight_layout()
plt.show()

#PS: Pressure, bar, 100 Hz --> 100 Messungen pro Sekunde

In [2]:
class DataPlotter:
    def __init__(self, dataset: dict):
        self.dataset = dataset
    
    def create_subplot(self):
        fig, axs = plt.subplots(6, 3, figsize=(15, 20))
        axes = axs.flatten()  
        for i, key in enumerate(self.dataset):
            ax = axes[i]
            ax.plot(self.dataset[key], color='blue', linewidth=1)
            ax.plot(np.mean(self.dataset[key], axis=1), color='red', linewidth=0.5)
            #ax.plot(np.var(self.dataset[key], axis=1), color='green')
            ax.set_title(key)
        plt.tight_layout()
        plt.show()
    
    def create_fft_plot(self):
        for key in self.dataset: 
            # Erstelle ein 2D-Array (z. B. eine Matrix)
            image = self.dataset[key]
            # 2D-Fourier-Transformation
            f_transform = np.fft.fft2(image)
            # Verschiebe den Nullfrequenzanteil in die Mitte des Spektrums
            f_transform_shifted = np.fft.fftshift(f_transform)
            # Berechne das Magnitudespektrum (Betrag)
            magnitude_spectrum = np.abs(f_transform_shifted)
            # Berechne das Phasenspektrum (Phase)
            phase_spectrum = np.angle(f_transform_shifted)
            # Summiere das Magnitudespektrum entlang der Spalten (horizontale Richtung)
            magnitude_spectrum_sum_cols = np.sum(magnitude_spectrum, axis=0)
            
            # Plots
            plt.figure(figsize=(12, 6))
            # Magnitudespektrum
            plt.subplot(1, 2, 1)
            plt.plot(magnitude_spectrum_sum_cols)
            plt.title(f"{key}: Magnitude Spectrum (Summe entlang der Spalten)")
            plt.xlabel("Spalten-Index")
            plt.ylabel("Summierte Magnitude")
            # Phasenspektrum
            plt.subplot(1, 2, 2)
            plt.imshow(phase_spectrum, cmap='twilight', aspect='auto')
            plt.title(f"{key}: Phase Spectrum")
            plt.colorbar(label='Phase (radians)')
            plt.tight_layout()
            plt.show()
            
plot = DataPlotter(data)

In [ ]:
# Umwandlung in Pandas-DataFrame
df = pd.DataFrame(data["eps1"])
df

In [ ]:
data_set = data["eps1"]
rolling_means_rows = np.array([
    pd.Series(row).rolling(window= 10).mean()
    for row in data_set
])


fig, ax = plt.subplots(figsize=(15, 6))
ax.plot(rolling_means_rows)
ax.set_title("Rolling Mean entlang der Zeilen")
plt.show()

Einzelne Signale plotten --> Ausreiser und Trends erkennen


In [ ]:
plot.create_subplot()

In [ ]:
plot.create_fft_plot()

In [8]:
#einzelne Signale in Means abspeichern
data_means = {}
for key in data:
    data_means[key] = np.mean(data[key], axis=1)

In [ ]:
fig, axs = plt.subplots(6, 3, figsize=(15, 20))
axes = axs.flatten()  
for i, key in enumerate(data_means):
    ax = axes[i]
    ax.plot(data_means[key], color='blue', linewidth=1)
    ax.set_title(key)
plt.tight_layout()
plt.show()

In [235]:
data_means_df = pd.DataFrame(data_means)

In [236]:
for column in data_means_df.columns:
    data_means_df[column] = data_means_df[column].rolling(window=10, min_periods=1).mean()

In [ ]:
for column in data_means_df.columns:
    data_means_df[column].plot(kind='line', title='Values Plot', xlabel='Index', ylabel='Values')
    plt.show()

In [98]:
class ManualFeatureExtractor:
    def __init__(self, dataset):
        self.dataset = dataset
        
    def extract_features(self):
        features = {}
        for i, key in enumerate(self.dataset, start=1):
            #features[f"{key}_{i}_mean"] = self.dataset[key].mean(axis=1)
            #features[f"{key}_{i}_median"] = np.median(self.dataset[key], axis=1)
            features[f"{key}_{i}_std"] = self.dataset[key].std(axis=1)
            #features[f"{key}_{i}_min"] = self.dataset[key].min(axis=1)
            #features[f"{key}_{i}_max"] = self.dataset[key].max(axis=1)
            #features[f"{key}_{i}_range"] = self.dataset[key].max(axis=1) - self.dataset[key].min(axis=1)
        self.df_features = pd.DataFrame(features)
        return self.df_features

extractor = ManualFeatureExtractor(data)
df_features = extractor.extract_features()

In [ ]:
print(df_features.shape, df_target.shape)

In [ ]:
nan_count_per_row = data_means_df.isna().sum(axis=1)
print(nan_count_per_row)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report

  
states = [27, 6728, 49122]
accs = []
features = df_features
target = df_target

for RANDOM_STATE in states:
    X_train, X_test, y_train, y_test = train_test_split(
        features, target, test_size = 0.2, random_state = RANDOM_STATE, stratify = target
    )
    
    model = AdaBoostClassifier(
    estimator=DecisionTreeClassifier(max_depth=1),
    n_estimators=50
    )
    
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    accs.append(accuracy_score(y_test, preds))
    print(f"Random State: {RANDOM_STATE}")
    print(classification_report(y_test, preds, zero_division=0.0))

accs_mean = round(np.mean(accs), 4)
accs_std = round(np.std(accs), 4)

print(f"Mean Accuracy: {accs_mean}")
print(f"Std Accuracy: {accs_std}")

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.signal import savgol_filter

# Beispiel: Lade dein 2D-Signal (2205 x 6000)
# Ersetze dies mit deinem tatsächlichen Signal
# signal = np.loadtxt('your_signal_file.csv')  # Falls als Datei vorliegt
signal = np.random.rand(2205, 6000) * 100  # Dummy-Daten zum Testen

# 1. Funktion: Ausreißer entfernen
def remove_outliers(data, threshold=3):
    """
    Entfernt Ausreißer anhand des Z-Scores.
    data: 2D-Array (Zeilen: Kanäle, Spalten: Zeitpunkte)
    threshold: Schwellenwert für Z-Score
    """
    z_scores = (data - np.mean(data, keepdims=True)) / np.std(data, keepdims=True)
    data_cleaned = np.where(np.abs(z_scores) > threshold, np.median(data, keepdims=True), data)
    return data_cleaned

# 2. Funktion: Gleitender Durchschnitt (Moving Average)
def smooth_signal(data, window_size=11):
    """
    Glättet das Signal mit einem gleitenden Durchschnitt.
    data: 2D-Array (Zeilen: Kanäle, Spalten: Zeitpunkte)
    window_size: Fenstergröße für den gleitenden Durchschnitt
    """
    return pd.DataFrame(data).rolling(window=window_size,min_periods=1).mean().values


# Pipeline zur Signalbereinigung
def clean_signal(signal):
    print("1. Entferne Ausreißer...")
    signal_cleaned = remove_outliers(signal)

    print("2. Glätte das Signal mit Moving Average...")
    signal_smoothed = smooth_signal(signal_cleaned, window_size=10)

    return signal_smoothed

# Bereinigung anwenden

data_cleaned
cleaned_signal = clean_signal(data_means['eps1'])

print(cleaned_signal)

# # Visualisierung des Original- und bereinigten Signals
# plt.figure(figsize=(20, 8))
# plt.subplot(2, 1, 1)
# plt.title("Originales Signal")
# plt.plot(signal, alpha=0.3)  # Transponieren für bessere Visualisierung'

plt.subplot(2, 1, 2)
plt.title("Bereinigtes Signal")
plt.plot(cleaned_signal, alpha=0.3)  # Transponieren für bessere Visualisierung
plt.tight_layout()
plt.show()


Merker für nach Urlaub: Datensätze von Sensoren, die mehrere Aufzeichnungen haben, zusammenführen, um einen großen Datensatz zu erhalten. Daraus dann wieder Standardabweichungen berechnen und in das Modell geben